# Libraries

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers accelerate

In [ ]:
import os, json
from datasets import Dataset, load_dataset
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [ ]:
%pip install transformers


In [ ]:
%pip install datasets



# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/data1

/content/drive/MyDrive/data1


In [ ]:
model_card = "distilbert-base-uncased"
device = 'cpu'

# Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_card)
model = AutoModelForQuestionAnswering.from_pretrained(model_card)
model


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [ ]:
model.to(device)
data_collator = DefaultDataCollator()

In [ ]:
def create_dataset(data_path = '/content/drive/MyDrive/data1/gems.json'):
    with open(data_path) as f:
        data = json.load(f)
    dataset = {}
    dataset['id'] = []
    dataset['title'] = []
    dataset['context'] = []
    dataset['question'] = []
    dataset['answers'] = []
    for d in data['data']:
        d = d['paragraphs'][0]
        for qa in d['qas']:
            context = d['context']
            question = qa['question']
            answer = qa['answers'][0]

            answer_start = answer['answer_start']
            answer_end = answer['answer_end']
            text = answer['text']
            answer = {
                    'answer_start': [answer_start],
                    'answer_end': [answer_end],
                    'text': [text]
                    }
            id_ = qa['id']
            title = 'gem-qna'

            dataset['id'].append(id_)
            dataset['title'].append(title)
            dataset['context'].append(context)
            dataset['question'].append(question)
            dataset['answers'].append(answer)

    dataset = Dataset.from_dict(dataset)
    return dataset

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
                    questions,
                    examples["context"],
                    max_length=384,
                    truncation="only_second",
                    return_offsets_mapping=True,
                    padding="max_length",
                    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs["offset_mapping"] = offset_mapping
    return inputs

In [ ]:
dataset = create_dataset()
dataset_processed = dataset.map(
                                preprocess_function,
                                remove_columns = dataset.column_names,
                                batched=True
                                )

Map:   0%|          | 0/2982 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
                                warmup_steps=500,
                                weight_decay=0.01,
                                logging_steps=500,
                                learning_rate=2e-5,
                                output_dir="models/gem-qna",
                                num_train_epochs=100,
                                save_strategy="epoch",
                                per_device_train_batch_size=10,
                                )
trainer = Trainer(
                model=model,
                args=training_args,
                tokenizer=tokenizer,
                train_dataset=dataset_processed,
                data_collator=data_collator,
                )

In [ ]:
%pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [ ]:
if not os.path.exists('/content/drive/MyDrive/data1/models/gem-qna-v6'):
  trainer.train()
  # save the model
  trainer.save_model('/content/drive/MyDrive/data1/models/gem-qna-v6')

model = '/content/drive/MyDrive/data1/models/gem-qna-v6'

Step,Training Loss


In [ ]:
from transformers import pipeline

In [ ]:
# load the model from pipeline
qa_pipeline = pipeline(
                        "question-answering",
                        model="models/gem-qna-v6",
                        device = -1
                        )

loading configuration file models/gem-qna\config.json
Model config DistilBertConfig {
  "_name_or_path": "models/gem-qna",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading configuration file models/gem-qna\config.json
Model config DistilBertConfig {
  "_name_or_path": "models/gem-qna",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model

In [ ]:
def get_context(question):
    with open('data/gems.json') as f:
        data = json.load(f)
        #context = None
        for item in data["data"]:
            for paragraph in item["paragraphs"]:
                for qa in paragraph["qas"]:
                    if qa["question"] == question:
                        context = paragraph["context"]
                        context
                        return context
    return None

In [ ]:
%pip install --upgrade tensorflow
%pip install --upgrade tensorflow-hub


     -------------------------------------- 276.5/276.5 MB 1.0 MB/s eta 0:00:00
     -------------------------------------- 430.6/430.6 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.2 MB/s eta 0:00:00
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
     ---------------------------------------- 5.6/5.6 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 kB 1.1 MB/s eta 0:00:00
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.1
    Uninstalling typing_extensions-4.6.1:
      Successfully uninstalled typing_extensions-4.6.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found exis

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\anaconda3\\envs\\myenv\\Lib\\site-packages\\google\\~rotobuf\\internal\\_api_implementation.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



     ---------------------------------------- 90.3/90.3 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.13.0
    Uninstalling tensorflow-hub-0.13.0:
      Successfully uninstalled tensorflow-hub-0.13.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Load the Universal Sentence Encoder's TF Hub module
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [ ]:
dataset = pd.read_json('data/gems.json')

In [ ]:
questions = []
for data_entry in dataset["data"]:
    for paragraph in data_entry["paragraphs"]:
        for qa in paragraph["qas"]:
            questions.append(qa["question"])


In [ ]:
# Compute a representation for each question
question_embeddings = model(questions)

In [ ]:
# Define a function to find the most similar question
def find_similar_question(query):
    query_embedding = model([query])[0]
    similarities = np.inner(question_embeddings, query_embedding)
    closest = np.argmax(similarities, axis=0)
    return questions[closest]


In [ ]:
# User input
user_input = "What are the gemstones suitable for new mothers?"

# Find and print the most similar question
similar_question = find_similar_question(user_input)

In [ ]:
# # Use the loaded model for prediction

context = get_context(similar_question)
qa_pipeline(question = similar_question, context = context)

{'score': 0.9999986886978149, 'start': 0, 'end': 11, 'answer': 'Rose Quartz'}

================

In [ ]:
# Assuming you have already loaded the dataset and created the qa_pipeline
# using the code you provided earlier

def calculate_exact_match(dataset):
    num_correct = 0
    total_examples = len(dataset)

    for i in range(total_examples):
        question = dataset["data"][i]["paragraphs"][0]["qas"][0]["question"]
        ground_truth_answer = dataset["data"][i]["paragraphs"][0]["qas"][0]["answers"][0]["text"]
        context = get_context(question)

        # Get the predicted answer from the pipeline
        prediction = qa_pipeline(question=question, context=context)

        # Extract the predicted answer text
        predicted_answer = prediction["answer"]

        # Calculate Exact Match
        if predicted_answer == ground_truth_answer:
            num_correct += 1

    exact_match = num_correct / total_examples
    return exact_match

# Call the function to calculate Exact Match
dataset = pd.read_json('data/gems.json')
exact_match = calculate_exact_match(dataset)

print("Exact Match (EM) on the dataset: {:.2%}".format(exact_match))


Exact Match (EM) on the dataset: 35.70%


In [ ]:
# Assuming you have already loaded the dataset and created the qa_pipeline
# using the code you provided earlier
from collections import Counter


def calculate_f1_score(dataset):
    total_f1 = 0
    total_examples = len(dataset)

    for i in range(total_examples):
        question = dataset["data"][i]["paragraphs"][0]["qas"][0]["question"]
        ground_truth_answer = dataset["data"][i]["paragraphs"][0]["qas"][0]["answers"][0]["text"]
        context = get_context(question)

        # Get the predicted answer from the pipeline
        prediction = qa_pipeline(question=question, context=context)

        # Extract the predicted answer text
        predicted_answer = prediction["answer"]

        # Calculate F1 score
        f1 = compute_f1(predicted_answer, ground_truth_answer)
        total_f1 += f1

    avg_f1 = total_f1 / total_examples
    return avg_f1

def compute_f1(pred_answer, ground_truth):
    common = Counter(pred_answer.split()) & Counter(ground_truth.split())
    num_same = sum(common.values())

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_answer.split())
    recall = 1.0 * num_same / len(ground_truth.split())
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Call the function to calculate F1 score
dataset = pd.read_json('data/gems.json')
f1_score = calculate_f1_score(dataset)

print("F1 Score on the dataset: {:.2%}".format(f1_score))


F1 Score on the dataset: 46.57%


In [ ]:
# Initialize counters for EM and F1 scores
total_cases = len(user_inputs)
em_matches = 0
total_precision = 0
total_recall = 0

for i in range(total_cases):
    user_input = user_inputs[i]
    ground_truth_answer = ground_truth_answers[i]
    predicted_answer = predict_answer(user_input)  # Use your QA model to predict the answer

    # Calculate EM score
    if predicted_answer == ground_truth_answer:
        em_matches += 1

    # Calculate precision and recall
    true_positive = int(predicted_answer == ground_truth_answer)
    false_positive = int(predicted_answer != ground_truth_answer)
    false_negative = int(predicted_answer != ground_truth_answer)

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    total_precision += precision
    total_recall += recall

# Calculate EM score
em_score = em_matches / total_cases

# Calculate average precision and recall
average_precision = total_precision / total_cases
average_recall = total_recall / total_cases

# Calculate F1 score
f1_score = 2 * (average_precision * average_recall) / (average_precision + average_recall)


-----------